In [10]:
# tools for handling files
import sys
import os

# pandas/numpy for handling data
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile

# for reading individual telomere length data from files
from ast import literal_eval

# for grabbing individual cells
import more_itertools

# my module containing functions for handling/visualizing/analyzing telomere length/chr rearrangement data
import telomere_methods_rad_patient as trp

# incase reloading modules is required
import importlib
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
&nbsp; 

...

&nbsp;

---

# Loading Telomere Length Data from TeloFISH
---

## Extracting telomere length data output from ImageJ from all radiation therapy patients

In [9]:
all_patients_dict = trp.generate_dictionary_from_TeloLength_data('../raw patient teloFISH data/')

SW9A non irrad.xlsx data extraction in progress..
BJ1 for SW9_.xlsx data extraction in progress..
SW11A non irrad.xlsx data extraction in progress..
BJ1 for SW15_.xlsx data extraction in progress..
SW6A non irrad.xlsx data extraction in progress..
SW6A irrad @ 4 Gy.xlsx data extraction in progress..
SW8B.xlsx data extraction in progress..
SW14A irrad @ 4 Gy.xlsx data extraction in progress..
SW8A irrad @ 4 Gy.xlsx data extraction in progress..
SW5A irrad @ 4 Gy.xlsx data extraction in progress..
SW8C.xlsx data extraction in progress..
SW1A non irrad.xlsx data extraction in progress..
BJ1 for SW11_.xlsx data extraction in progress..
SW16A non irrad.xlsx data extraction in progress..
BJ1 for SW13_.xlsx data extraction in progress..
BJ-hTERT for SW9_.xlsx data extraction in progress..
BJ1 for SW14_.xlsx data extraction in progress..
SW9B.xlsx data extraction in progress..
BJ1 for SW8_.xlsx data extraction in progress..
SW_1_ok_3_C_.xlsx data extraction in progress..


/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:2275: RuntimeWarning: Mean of empty slice.
  mns = a.mean(axis=axis)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:207: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


SW3A irrad @ 4 Gy.xlsx data extraction in progress..
SW11A irrad @ 4 Gy.xlsx data extraction in progress..
BJ1 for SW16_.xlsx data extraction in progress..
BJ1 for SW12_.xlsx data extraction in progress..
SW8A non irrad.xlsx data extraction in progress..
BJ-hTERT for SW8_.xlsx data extraction in progress..
SW10A non irrad.xlsx data extraction in progress..
SW12A irrad @ 4 Gy.xlsx data extraction in progress..
SW9C.xlsx data extraction in progress..
BJ1 for SW10_.xlsx data extraction in progress..
SW7A non irrad.xlsx data extraction in progress..
SW1A irrad @ 4 Gy.xlsx data extraction in progress..
SW13A irrad @ 4 Gy.xlsx data extraction in progress..
SW1B.xlsx data extraction in progress..
BJ-hTERT for SW6_.xlsx data extraction in progress..
SW13B.xlsx data extraction in progress..
BJ1 for SW2_.xlsx data extraction in progress..
SW2A non irrad.xlsx data extraction in progress..
SW5C.xlsx data extraction in progress..
SW15C.xlsx data extraction in progress..
SW7C.xlsx data extraction in

## Making dataframe from dict w/ all patients telomere length data, contains telo means & individual telos as list

In [11]:
all_patients_df = trp.generate_dataframe_from_dict(all_patients_dict)

# don't need telo means per cell @ this time
all_patients_df = all_patients_df.drop(['cell data'], axis=1)
print(all_patients_df.shape)

To display graphs pass the value "yes graphs" to the function otherwise default option="no graphs"
(59, 7)


## Saving all patients telomere length data for later retrieval

In [84]:
# changing telo data to list in prep for saving to csv
all_patients_df['telo data'] = all_patients_df['telo data'].apply(lambda row: row.tolist())
all_patients_df.to_csv('../compiled patient data csv files/all_patients_df.csv', index=False)

## Generating all patients telo df containing telo counts per quartile melted into tidy data format 

In [85]:
melted_all_patients_df = pd.melt(
    all_patients_df,
    id_vars = [col for col in all_patients_df.columns if col != 'Q1' and col != 'Q2-3' and col != 'Q4'],
    var_name='relative Q',
    value_name='Q freq counts')

melted_all_patients_df['Q freq counts'] = melted_all_patients_df['Q freq counts'].astype('float64')
melted_all_patients_df.head(4)

,patient id,timepoint,telo data,chr data,status,telo means,relative Q,Q freq counts
0,1,1 non irrad,"[141.48603338585482, 96.08283795389858, 69.687...",chr data,IT WORKS PEGGY <333,84.796483,Q1,1195.0
1,1,2 irrad @ 4 Gy,"[52.79276810653768, 100.3062594024216, 111.921...",chr data,IT WORKS PEGGY <333,90.975826,Q1,724.0
2,1,3 B,"[127.75977641558534, 210.11731823720234, 149.9...",chr data,IT WORKS PEGGY <333,116.779989,Q1,231.0
3,1,4 C,"[108.75310229946761, 73.90987534915276, 93.971...",chr data,IT WORKS PEGGY <333,99.346299,Q1,372.0


## Saving melted all patients df to csv

In [86]:
melted_all_patients_df.to_csv('../compiled patient data csv files/melted_all_patients_df.csv', index=False)

## Pivoted Dataframe w/ timepoints as columns, and telomere length means for each patient timepoint in rows

In [87]:
pivot_patients_telo_means_df = all_patients_df.pivot(index='patient id', columns='timepoint', values='telo means')
pivot_patients_telo_means_df = pivot_patients_telo_means_df.drop(13)
pivot_patients_telo_means_df.head(4)

timepoint,1 non irrad,2 irrad @ 4 Gy,3 B,4 C
patient id,,,,
1,84.796483,90.975826,116.779989,99.346299
2,119.773675,133.199443,159.827558,108.915327
3,83.350928,87.295453,101.432564,95.669501
5,85.506106,113.095980,118.340459,97.832190


## Saving pivoted telo means df to file

In [88]:
pivot_patients_telo_means_df.to_csv('../compiled patient data csv files/pivot_patients_telo_means_df.csv', index=False)

## Exploding individual telomere length measurements from contained list into dataframe (i.e row per individual telomere) while retaining related column info

In [89]:
# can imagine the lists containing the individual telos per patient exploding to the right; maintains the index relationship
explode_telos_raw = all_patients_df['telo data'].apply(pd.Series)

print(explode_telos_raw.shape)
explode_telos_raw.head(4)

(59, 4600)


,0,1,2,3,4,5,6,7,8,9,...,4590,4591,4592,4593,4594,4595,4596,4597,4598,4599
0,141.486033,96.082838,69.687151,59.128492,133.037776,115.088234,57.016760,83.412574,98.195531,65.463032,...,114.033519,84.468429,58.072045,60.184357,159.434160,69.687151,34.843575,47.513966,100.307262,60.184357
1,52.792768,100.306259,111.921788,88.606283,85.524284,100.306259,147.821229,51.737430,79.189944,69.687151,...,112.976524,131.983240,117.201117,69.687151,81.301676,122.480447,114.033519,115.089385,111.921788,129.871508
2,127.759776,210.117318,149.931461,103.473825,98.195531,116.145251,97.139665,111.921788,137.262570,86.581006,...,95.027933,104.530726,125.646788,116.145251,85.525140,138.318436,146.765363,67.575419,142.541899,138.318436
3,108.753102,73.909875,93.971127,91.860335,77.078212,92.916201,45.402235,122.480447,137.262570,107.698324,...,160.490015,101.362115,73.910614,128.814354,108.753102,81.301676,115.088234,109.810056,59.128492,87.636871


In [90]:
exploded_telos_all_patients_df = (explode_telos_raw
                                  
    # we'll merge the exploded telos df w/ our original all patients df on the index!
    .merge(all_patients_df, right_index = True, left_index = True)
    .drop(['telo data', 'Q1', 'Q2-3', 'Q4'], axis = 1)
    .melt(id_vars = ['patient id', 'timepoint', 'telo means'], value_name = "individual telomeres") 
    .drop("variable", axis = 1)
    .dropna())

# exploded_telos_all_patients_df['telo data exploded'] = exploded_telos_all_patients_df['telo data exploded'].astype('float64')
exploded_telos_all_patients_df.head(4)

,patient id,timepoint,chr data,status,telo means,individual telomeres
0,1,1 non irrad,chr data,IT WORKS PEGGY <333,84.796483,141.486033
1,1,2 irrad @ 4 Gy,chr data,IT WORKS PEGGY <333,90.975826,52.792768
2,1,3 B,chr data,IT WORKS PEGGY <333,116.779989,127.759776
3,1,4 C,chr data,IT WORKS PEGGY <333,99.346299,108.753102


## Saving exploded telomere df for later retrieval

In [91]:
exploded_telos_all_patients_df.to_csv('../compiled patient data csv files/exploded_telos_all_patients_df.csv', index=False)

# Loading Telomere Length Data from qPCR - Aidan & Lynn
---

## Extracting average telomere length data by qPCR data from Aidan & Lynn

In [8]:
all_qPCR_df = pd.read_excel('../qPCR telo data/REVISED Tel +Alb (both plates) 2019-08-05 13 Quantification Cq Results.xlsx',
                            sheet_name=1,
                           skiprows=1,
                            usecols=[24, 25, 26,]
                           )

all_qPCR_df.columns = ['Sample', 'telo means qPCR', 'SEM']
all_qPCR_df['Sample'] = all_qPCR_df['Sample'].astype('str')

In [9]:
all_qPCR_df.dropna(axis=0, inplace=True)
all_qPCR_df.drop([24, 47, 48], axis=0, inplace=True)

In [11]:
all_qPCR_df['Sample'] = all_qPCR_df['Sample'].apply(lambda x: trp.change_sample_ID(x))
all_qPCR_df['timepoint'] = all_qPCR_df['Sample'].apply(lambda x: trp.make_timepoint_col(x))
all_qPCR_df['patient id'] = all_qPCR_df['Sample'].apply(lambda x: trp.make_patient_ID(x))

all_qPCR_df['patient id'] = all_qPCR_df['patient id'].astype('str')
all_qPCR_df.drop(['Sample'], axis=1, inplace=True)

In [12]:
cols = ['patient id', 'timepoint', 'telo means qPCR', 'SEM']
all_qPCR_df = all_qPCR_df.reindex(columns=cols)
all_qPCR_df.head(4)

,patient id,timepoint,telo means qPCR,SEM
0,1,1 non irrad,2.182028,0.083233
1,1,3 B,1.964348,0.051942
2,1,4 C,1.749799,0.068195
3,2,1 non irrad,1.669445,0.034776


In [13]:
all_qPCR_df.to_csv('../qPCR telo data/all_qPCR_df.csv', index=False)

# Loading Chromosome Aberration Data from Subtelo-dGH 
---

In [39]:
all_chr_aberr_df = trp.make_dataframe_chr_aberr_data('../dGH scoresheets/')

<DirEntry 'SW14_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW3_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW8_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW13_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW9_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW5_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW12_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW15_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW2_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW10_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW7_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW1_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW16_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW11_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW6_timepoints_subtelodGH.xlsx'>


In [40]:
all_chr_aberr_df.to_csv('../compiled patient data csv files/all_chr_aberr_df.csv', index=False)